In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
import PIL
from PIL import Image
import os
import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
Path = 'drive/MyDrive'
path1 = 'drive/MyDrive/data/valid'
path = 'drive/MyDrive/data/train'
patht = 'train'
pathv = 'valid'

In [ ]:
data_train = np.empty((1000, 128, 128, 10))
label_train = np.empty((1000, 128, 128, 10))

In [ ]:

data_valid = np.empty((400, 128, 128, 10))
label_valid = np.empty((400, 128, 128, 10))

NameError: ignored

In [ ]:
import random
files = os.listdir(os.path.join(path, 'blurred'))
files1 = os.listdir(os.path.join(path1, 'blurred'))
print(len(files))
print(len(files1))
random.shuffle(files)

1782
315


In [ ]:
from IPython.display import clear_output
import shutil
for i in range(1248):
  clear_output(wait = True)
  print('train:', str(i) + '/' + str(1247))
  shutil.copy(os.path.join(path, 'blurred', files[i]), os.path.join(patht, 'blurred', files[i]))
  shutil.copy(os.path.join(path, 'orig', files[i]), os.path.join(patht, 'orig', files[i]))


train: 1247/1247


In [ ]:
for i in range(200):
  clear_output(wait = True)
  print('valid:', str(i) + '/' + str(315))a
  shutil.copy(os.path.join(path1, 'blurred', files1[i]), os.path.join(pathv, 'blurred', files1[i]))
  shutil.copy(os.path.join(path1, 'orig', files1[i]), os.path.join(pathv, 'orig', files1[i]))

valid: 199/315


In [ ]:
from IPython.display import clear_output
for i in range(1000):
  clear_output(wait = True)
  print('train:', str(i) + '/' + str(500))
  data = np.load(os.path.join(patht, 'blurred', files[i])).astype(np.float16)
  label = np.load(os.path.join(patht, 'orig', files[i])).astype(np.float16)
  if i == 17:
    data = np.load(os.path.join(patht, 'blurred', files[i - 1])).astype(np.float16)
    label = np.load(os.path.join(patht, 'orig', files[i - 1])).astype(np.float16)
  j = random.randint(0, data.shape[0] - 1)
  data_train[i] = data[j].transpose(1, 2, 0)
  label_train[i] = label[j].transpose(1, 2, 0)

train: 999/500


In [ ]:
for i in range(200):
  clear_output(wait = True)
  print('valid:', str(i) + '/' + str(500))
  data = np.load(os.path.join(pathv, 'blurred', files1[i])).astype(np.float16)
  label = np.load(os.path.join(pathv, 'orig', files1[i])).astype(np.float16)
  j = random.randint(0, data.shape[0] - 1)
  data_valid[i * 2] = data[j].transpose(1, 2, 0)
  label_valid[i * 2] = label[j].transpose(1, 2, 0)
  j = random.randint(0, data.shape[0] - 1)
  data_valid[i * 2 + 1] = data[j].transpose(1, 2, 0)
  label_valid[i * 2 + 1] = label[j].transpose(1, 2, 0)

valid: 199/500


In [ ]:
np.save('label_train.npy', label_train)
np.save('label_valid.npy', label_valid)

In [ ]:
inp = tf.keras.layers.Input((128, 128,10))
x0 = tf.keras.layers.Conv2D(256,3,padding = 'same')(inp)
x = x0
for i in range(4):
    y = tf.keras.layers.Conv2D(256,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256,3,padding = 'same', activation = 'relu')(x)
x = tf.keras.layers.Add()([x0,x])
x = tf.keras.layers.Conv2D(160,3,padding = 'same')(x)
x = tf.keras.layers.Lambda (lambda z: tf.nn.depth_to_space(z, 4))(x)
x = tf.keras.layers.AveragePooling2D(4)(x)
out = x
model1 = tf.keras.models.Model(inputs = inp, outputs = out)

In [ ]:
print(model1.summary())

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                0)]                                                               
                                                                                                  
 conv2d_500 (Conv2D)            (None, 128, 128, 25  23296       ['input_5[0][0]']                
                                6)                                                                
                                                                                                  
 conv2d_501 (Conv2D)            (None, 128, 128, 25  590080      ['conv2d_500[0][0]']             
                                6)                                                          

In [ ]:
inp = tf.keras.layers.Input((None, None,10))
x0 = tf.keras.layers.Conv2D(256,3,padding = 'same')(inp)
x = x0
for i in range(2):
    y = tf.keras.layers.Conv2D(256,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256, 1, activation = 'relu')(x)
for i in range(2):
    y = tf.keras.layers.Conv2D(256,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256,3,padding = 'same', activation = 'relu')(x)
x = tf.keras.layers.Add()([x0,x])
x = tf.keras.layers.Conv2D(160,3,padding = 'same', activation = 'sigmoid')(x)
x = tf.keras.layers.Lambda (lambda z: tf.nn.depth_to_space(z, 4))(x)
x = tf.keras.layers.AveragePooling2D(4)(x)
out = x
model2 = tf.keras.models.Model(inputs = inp, outputs = out)

In [ ]:
print(model2.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 10)]                                                             
                                                                                                  
 conv2d_11 (Conv2D)             (None, None, None,   23296       ['input_2[0][0]']                
                                256)                                                              
                                                                                                  
 conv2d_12 (Conv2D)             (None, None, None,   590080      ['conv2d_11[0][0]']              
                                256)                                                        

In [ ]:
inp = tf.keras.layers.Input((None, None,10))
x0 = tf.keras.layers.Conv2D(256,3,padding = 'same')(inp)
x = x0
for i in range(4):
    y = tf.keras.layers.Conv2D(256,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
#x = tf.keras.layers.Conv2D(256, 1, activation = 'relu')(x)
for i in range(4):
    y = tf.keras.layers.Conv2D(256,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256,3,padding = 'same', activation = 'relu')(x)
x = tf.keras.layers.Add()([x0,x])
x = tf.keras.layers.Conv2D(160,3,padding = 'same', activation = 'sigmoid')(x)
x = tf.keras.layers.Lambda (lambda z: tf.nn.depth_to_space(z, 4))(x)
x = tf.keras.layers.AveragePooling2D(4)(x)
out = x
model3 = tf.keras.models.Model(inputs = inp, outputs = out)

In [ ]:
print(model3.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None, None,  0           []                               
                                 10)]                                                             
                                                                                                  
 conv2d_23 (Conv2D)             (None, None, None,   23296       ['input_3[0][0]']                
                                256)                                                              
                                                                                                  
 conv2d_24 (Conv2D)             (None, None, None,   590080      ['conv2d_23[0][0]']              
                                256)                                                        

In [ ]:
inp = tf.keras.layers.Input((None, None,10))
x0 = tf.keras.layers.Conv2D(256,3,padding = 'same')(inp)
x = x0
for i in range(5):
    y = tf.keras.layers.Conv2D(256,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256, 1, activation = 'relu')(x)
for i in range(5):
    y = tf.keras.layers.Conv2D(256,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256,3,padding = 'same', activation = 'relu')(x)
x = tf.keras.layers.Add()([x0,x])
x = tf.keras.layers.Conv2D(160,3,padding = 'same', activation = 'sigmoid')(x)
x = tf.keras.layers.Lambda (lambda z: tf.nn.depth_to_space(z, 4))(x)
x = tf.keras.layers.AveragePooling2D(4)(x)
out = x
model4 = tf.keras.models.Model(inputs = inp, outputs = out)

In [ ]:
print(model4.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, None, None,  0           []                               
                                 10)]                                                             
                                                                                                  
 conv2d_43 (Conv2D)             (None, None, None,   23296       ['input_4[0][0]']                
                                256)                                                              
                                                                                                  
 conv2d_44 (Conv2D)             (None, None, None,   590080      ['conv2d_43[0][0]']              
                                256)                                                        

In [ ]:
inp = tf.keras.layers.Input((None, None,10))
x0 = tf.keras.layers.Conv2D(256,3,padding = 'same')(inp)
x0 = tf.keras.layers.Conv2D(64,3,padding = 'same')(x0)
x = x0
for i in range(10):
    y = tf.keras.layers.Conv2D(64,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(64,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(64, 1, activation = 'relu')(x)
for i in range(10):
    y = tf.keras.layers.Conv2D(64,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(64,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(64,3,padding = 'same', activation = 'relu')(x)
x = tf.keras.layers.Add()([x0,x])
x = tf.keras.layers.Conv2D(40,3,padding = 'same', activation = 'sigmoid')(x)
x = tf.keras.layers.Lambda (lambda z: tf.nn.depth_to_space(z, 2))(x)
x = tf.keras.layers.AveragePooling2D(2)(x)
out = x
model11 = tf.keras.models.Model(inputs = inp, outputs = out)

In [ ]:
print(model11.summary())

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None, None,  0           []                               
                                 10)]                                                             
                                                                                                  
 conv2d_67 (Conv2D)             (None, None, None,   23296       ['input_5[0][0]']                
                                256)                                                              
                                                                                                  
 conv2d_68 (Conv2D)             (None, None, None,   147520      ['conv2d_67[0][0]']              
                                64)                                                         

In [ ]:
inp = tf.keras.layers.Input((None, None,10))
x0 = tf.keras.layers.Conv2D(256,3,padding = 'same')(inp)
x0 = tf.keras.layers.Conv2D(64,3,padding = 'same')(x0)
x = x0
for i in range(30):
    y = tf.keras.layers.Conv2D(64,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(64,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(64, 1, activation = 'relu')(x)
for i in range(30):
    y = tf.keras.layers.Conv2D(64,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(64,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(64,3,padding = 'same', activation = 'relu')(x)
x = tf.keras.layers.Add()([x0,x])
x = tf.keras.layers.Conv2D(40,3,padding = 'same', activation = 'sigmoid')(x)
x = tf.keras.layers.Lambda (lambda z: tf.nn.depth_to_space(z, 2))(x)
x = tf.keras.layers.AveragePooling2D(2)(x)
out = x
model12 = tf.keras.models.Model(inputs = inp, outputs = out)

In [ ]:
print(model12.summary())

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, None, None,  0           []                               
                                 10)]                                                             
                                                                                                  
 conv2d_237 (Conv2D)            (None, None, None,   23296       ['input_7[0][0]']                
                                256)                                                              
                                                                                                  
 conv2d_238 (Conv2D)            (None, None, None,   147520      ['conv2d_237[0][0]']             
                                64)                                                         

In [ ]:
!pip install pytorch-msssim

In [ ]:
#from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
from tensorflow.keras import optimizers
from tensorflow.image import psnr

def mssim(X, Y):
  return tf.image.ssim_multiscale(X, Y, 1, filter_size = 6)

def lssim(X, Y):
  return 1 - tf.image.ssim_multiscale(X, Y, 1, filter_size = 6)

def psnr1(X, Y):
  return psnr(X, Y, 1)

model1.compile(loss = lssim, metrics = [mssim, psnr1], optimizer = optimizers.Adam(learning_rate = 1e-5))


In [ ]:
model2.compile(loss = lssim, metrics = [mssim, psnr1], optimizer = optimizers.Adam(learning_rate = 1e-5))
model3.compile(loss = lssim, metrics = [mssim, psnr1], optimizer = optimizers.Adam(learning_rate = 1e-5))
model4.compile(loss = lssim, metrics = [mssim, psnr1], optimizer = optimizers.Adam(learning_rate = 1e-5))
model11.compile(loss = lssim, metrics = [mssim, psnr1], optimizer = optimizers.Adam(learning_rate = 1e-5))
#model12.compile(loss = lssim, metrics = [mssim, psnr1], optimizer = optimizers.Adam(learning_rate = 1e-8, epsilon = 1e-9, clipnorm = 1))

In [ ]:
p = 'drive/MyDrive/data'
data_train = np.load(os.path.join(p, 'dt.npy')).astype(np.float16)
label_train = np.load(os.path.join(p, 'lt.npy')).astype(np.float16)
data_valid = np.load(os.path.join(p, 'dv.npy')).astype(np.float16)
label_valid = np.load(os.path.join(p, 'lv.npy')).astype(np.float16)

In [ ]:
print(label_train.shape, label_valid.shape)
print(data_train.shape, data_valid.shape)

(996, 128, 128, 10) (400, 128, 128, 10)
(996, 128, 128, 10) (400, 128, 128, 10)


In [ ]:
print(data_train[0][0])

[[ 4.7180e-02  1.6830e-02  4.7028e-02 ...  7.9102e-02  6.1951e-02
   4.4220e-02]
 [ 1.6089e-01  1.5796e-01  1.5796e-01 ...  1.8250e-01  1.7676e-01
   1.6296e-01]
 [ 2.2363e-01  1.7212e-01  2.2522e-01 ...  3.0054e-01  3.0884e-01
   2.8613e-01]
 ...
 [ 1.3741e-02  5.9485e-05  2.9266e-02 ...  1.0181e-01  5.3009e-02
   4.8767e-02]
 [ 2.7664e-02 -1.1894e-02  2.3346e-02 ...  8.0627e-02  6.7200e-02
   3.9062e-02]
 [ 2.1194e-02  1.2161e-02  1.9135e-02 ...  6.4758e-02  4.6600e-02
   3.5645e-02]]


In [ ]:
data_train = data_train / np.amax(data_train)
data_valid = data_valid / np.amax(data_valid)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((data_train, label_train))
#train_dataset = train_dataset.batch(32)
valid_dataset = tf.data.Dataset.from_tensor_slices((data_valid, label_valid))
#valid_dataset = valid_dataset.batch(32)

In [ ]:
count1 = 0
import math
count2 = 0
for i in range(data_train.shape[0]):
  if i < data_train.shape[0] and math.isnan(np.amax(data_train[i])) or math.isnan(np.amax(label_train[i])):
    data_train = np.delete(data_train, i, axis = 0)
    count1 += 1
    label_train = np.delete(label_train, i, axis = 0)
print(count1)
for i in range(data_valid.shape[0]):
  if i < data_valid.shape[0] and math.isnan(np.amax(data_valid[i])) or math.isnan(np.amax(label_valid[i])):
    data_valid = np.delete(data_valid, i, axis = 0)
    count2 += 1
    label_valid = np.delete(label_valid, i, axis = 0)
print(count2)
print(data_train.shape)
print(data_valid.shape)

0
0
(996, 128, 128, 10)
(400, 128, 128, 10)


In [ ]:

print(np.amax(data_valid))

1.0


In [ ]:
p = 'drive/MyDrive/data'
tf.data.experimental.save(train_dataset, os.path.join(p, 'train'))
tf.data.experimental.save(valid_dataset, os.path.join(p, 'valid'))
np.save(np.array(data_train, label_train), os.path.join(p, 't.npy'))
np.save(np.array(data_valid, label_valid), os.path.join(p, 'v.npy'))

TypeError: ignored

In [ ]:
p = 'drive/MyDrive/data'
train_dataset = tf.data.experimental.load(os.path.join(p, 'train'))
valid_dataset = tf.data.experimental.load(os.path.join(p, 'valid'))

In [ ]:
train_dataset = train_dataset.batch(8)
valid_dataset = valid_dataset.batch(8)

In [ ]:
print(train_dataset)

<BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 10), dtype=tf.float16, name=None), TensorSpec(shape=(None, 128, 128, 10), dtype=tf.float16, name=None))>


In [ ]:
model1.fit(train_dataset, epochs = 50, 
           validation_data = valid_dataset, verbose = 1, validation_freq = 10)

Epoch 1/50
125/125 [==============================] - 47s 312ms/step - loss: 0.3888 - mssim: 0.6112 - psnr1: 11.4642
Epoch 2/50
125/125 [==============================] - 38s 307ms/step - loss: 0.1296 - mssim: 0.8704 - psnr1: 21.2523
Epoch 3/50
125/125 [==============================] - 38s 306ms/step - loss: 0.1089 - mssim: 0.8911 - psnr1: 23.0739
Epoch 4/50
125/125 [==============================] - 38s 307ms/step - loss: 0.1024 - mssim: 0.8976 - psnr1: 23.7963
Epoch 5/50
125/125 [==============================] - 38s 306ms/step - loss: 0.0990 - mssim: 0.9010 - psnr1: 24.1946
Epoch 6/50
125/125 [==============================] - 38s 306ms/step - loss: 0.0970 - mssim: 0.9030 - psnr1: 24.4490
Epoch 7/50
125/125 [==============================] - 38s 306ms/step - loss: 0.0955 - mssim: 0.9045 - psnr1: 24.6327
Epoch 8/50
125/125 [==============================] - 38s 306ms/step - loss: 0.0944 - mssim: 0.9056 - psnr1: 24.7825
Epoch 9/50
125/125 [==============================] - 38s 306ms/

In [ ]:
model1.save('drive/MyDrive/models/model1')

INFO:tensorflow:Assets written to: drive/MyDrive/models/model1/assets


In [ ]:
model2.fit(train_dataset, epochs = 50, validation_data = valid_dataset)

Epoch 1/50
125/125 [==============================] - 57s 390ms/step - loss: 0.4468 - mssim: 0.5532 - psnr1: 10.3318 - val_loss: 0.3507 - val_mssim: 0.6493 - val_psnr1: 8.8768
Epoch 2/50
125/125 [==============================] - 46s 369ms/step - loss: 0.2205 - mssim: 0.7795 - psnr1: 11.2545 - val_loss: 0.3011 - val_mssim: 0.6989 - val_psnr1: 11.3462
Epoch 3/50
125/125 [==============================] - 46s 367ms/step - loss: 0.1584 - mssim: 0.8416 - psnr1: 17.0152 - val_loss: 0.2246 - val_mssim: 0.7754 - val_psnr1: 20.0420
Epoch 4/50
125/125 [==============================] - 46s 367ms/step - loss: 0.1288 - mssim: 0.8712 - psnr1: 22.1519 - val_loss: 0.1844 - val_mssim: 0.8156 - val_psnr1: 22.5265
Epoch 5/50
125/125 [==============================] - 46s 368ms/step - loss: 0.1220 - mssim: 0.8780 - psnr1: 22.7828 - val_loss: 0.1646 - val_mssim: 0.8354 - val_psnr1: 23.2094
Epoch 6/50
125/125 [==============================] - 46s 368ms/step - loss: 0.1177 - mssim: 0.8823 - psnr1: 23.0654

In [ ]:
model2.save('drive/MyDrive/models/model2')

INFO:tensorflow:Assets written to: drive/MyDrive/models/model2/assets


In [ ]:
model3.fit(train_dataset, epochs = 50, validation_data = valid_dataset)

Epoch 1/50
125/125 [==============================] - 90s 654ms/step - loss: 0.3456 - mssim: 0.6544 - psnr1: 10.6584 - val_loss: 0.3174 - val_mssim: 0.6826 - val_psnr1: 10.4603
Epoch 2/50
125/125 [==============================] - 79s 632ms/step - loss: 0.1662 - mssim: 0.8338 - psnr1: 16.2428 - val_loss: 0.2165 - val_mssim: 0.7835 - val_psnr1: 20.6141
Epoch 3/50
125/125 [==============================] - 79s 632ms/step - loss: 0.1255 - mssim: 0.8745 - psnr1: 22.0946 - val_loss: 0.1685 - val_mssim: 0.8315 - val_psnr1: 23.2405
Epoch 4/50
125/125 [==============================] - 79s 632ms/step - loss: 0.1158 - mssim: 0.8842 - psnr1: 22.9531 - val_loss: 0.1437 - val_mssim: 0.8563 - val_psnr1: 23.5762
Epoch 5/50
125/125 [==============================] - 79s 632ms/step - loss: 0.1108 - mssim: 0.8892 - psnr1: 23.4339 - val_loss: 0.1320 - val_mssim: 0.8680 - val_psnr1: 23.2817
Epoch 6/50
125/125 [==============================] - 79s 632ms/step - loss: 0.1074 - mssim: 0.8926 - psnr1: 23.740

In [ ]:
model3.save('drive/MyDrive/models/model3')

INFO:tensorflow:Assets written to: drive/MyDrive/models/model3/assets


In [ ]:

model12.compile(loss = lssim, metrics = [mssim, psnr], optimizer = optimizers.Adam(learning_rate = 1.13e-5))
model12.fit(train_dataset, epochs = 100, validation_data = valid_dataset, validation_freq = 1)

In [ ]:
lr = 0.9e-5
for i in range(100):
  model12.compile(loss = lssim, metrics = [mssim, psnr], optimizer = optimizers.Adam(learning_rate = lr))
  print(lr)
  model12.fit(train_dataset, epochs = i + 1, initial_epoch = i, validation_data = valid_dataset)
  lr += 0.01e-5

In [ ]:
model12.save('drive/aMyDrive/models/model12')

In [ ]:
model4.fit(train_dataset, epochs = 50, validation_data = valid_dataset, validation_freq = 10)

Epoch 1/50
125/125 [==============================] - 90s 659ms/step - loss: 0.3394 - mssim: 0.6606 - psnr1: 11.1768
Epoch 2/50
125/125 [==============================] - 82s 658ms/step - loss: 0.1563 - mssim: 0.8437 - psnr1: 17.5905
Epoch 3/50
125/125 [==============================] - 82s 658ms/step - loss: 0.1214 - mssim: 0.8786 - psnr1: 22.5299
Epoch 4/50
125/125 [==============================] - 82s 659ms/step - loss: 0.1124 - mssim: 0.8876 - psnr1: 23.2937
Epoch 5/50
125/125 [==============================] - 82s 658ms/step - loss: 0.1076 - mssim: 0.8924 - psnr1: 23.6966
Epoch 6/50
125/125 [==============================] - 82s 659ms/step - loss: 0.1041 - mssim: 0.8959 - psnr1: 24.0283
Epoch 7/50
125/125 [==============================] - 82s 659ms/step - loss: 0.1015 - mssim: 0.8985 - psnr1: 24.2992
Epoch 8/50
125/125 [==============================] - 82s 659ms/step - loss: 0.0995 - mssim: 0.9005 - psnr1: 24.4621
Epoch 9/50
125/125 [==============================] - 82s 659ms/

In [ ]:
model4.save('drive/MyDrive/models/model4')

INFO:tensorflow:Assets written to: drive/MyDrive/models/model4/assets


In [ ]:
model11.fit(train_dataset, epochs = 50, validation_data = valid_dataset, validation_freq = 10)

Epoch 1/50
125/125 [==============================] - 32s 180ms/step - loss: 0.4284 - mssim: 0.5716 - psnr1: 10.5862
Epoch 2/50
125/125 [==============================] - 22s 178ms/step - loss: 0.2371 - mssim: 0.7629 - psnr1: 12.9947
Epoch 3/50
125/125 [==============================] - 22s 178ms/step - loss: 0.1899 - mssim: 0.8101 - psnr1: 14.8523
Epoch 4/50
125/125 [==============================] - 22s 178ms/step - loss: 0.1675 - mssim: 0.8325 - psnr1: 16.6426
Epoch 5/50
125/125 [==============================] - 22s 178ms/step - loss: 0.1564 - mssim: 0.8436 - psnr1: 17.3829
Epoch 6/50
125/125 [==============================] - 22s 178ms/step - loss: 0.1499 - mssim: 0.8501 - psnr1: 17.7120
Epoch 7/50
125/125 [==============================] - 22s 178ms/step - loss: 0.1452 - mssim: 0.8548 - psnr1: 17.8952
Epoch 8/50
125/125 [==============================] - 22s 178ms/step - loss: 0.1415 - mssim: 0.8585 - psnr1: 18.0251
Epoch 9/50
125/125 [==============================] - 22s 178ms/

In [ ]:
model11.save('drive/MyDrive/models/model11')

INFO:tensorflow:Assets written to: drive/MyDrive/models/model11/assets


In [ ]:
inp = tf.keras.layers.Input((None, None,10))
x0 = tf.keras.layers.Conv2D(256,3,padding = 'same')(inp)
x = x0
for i in range(5):
    y = tf.keras.layers.Conv2D(256,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256, 1, activation = 'relu')(x)
for i in range(5):
    y = tf.keras.layers.Conv2D(256,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256,3,padding = 'same', activation = 'relu')(x)
x = tf.keras.layers.Add()([x0,x])
x = tf.keras.layers.Conv2D(160,3,padding = 'same', activation = 'relu')(x)
x = tf.keras.layers.Lambda (lambda z: tf.nn.depth_to_space(z, 4))(x)
x = tf.keras.layers.AveragePooling2D(4)(x)
out = x
model4_1 = tf.keras.models.Model(inputs = inp, outputs = out)

In [ ]:
inp = tf.keras.layers.Input((None, None,10))
x0 = tf.keras.layers.Conv2D(256,3,padding = 'same')(inp)
x = x0
for i in range(5):
    y = tf.keras.layers.Conv2D(256,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256, 1, activation = 'relu')(x)
for i in range(5):
    y = tf.keras.layers.Conv2D(256,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256,3,padding = 'same', activation = 'relu')(x)
x = tf.keras.layers.Add()([x0,x])
x = tf.keras.layers.Conv2D(160,3,padding = 'same', activation = 'sigmoid')(x)
x = tf.keras.layers.Lambda (lambda z: tf.nn.depth_to_space(z, 4))(x)
x = tf.keras.layers.AveragePooling2D(4)(x)
out = x
model4_2 = tf.keras.models.Model(inputs = inp, outputs = out)

In [ ]:
from tensorflow.keras import optimizers
from tensorflow.image import psnr

def mssim(X, Y):
  return tf.image.ssim_multiscale(X, Y, 1, filter_size = 6)

def lssim(X, Y):
  return 1 - tf.image.ssim_multiscale(X, Y, 1, filter_size = 6)

def psnr1(X, Y):
  return psnr(X, Y, 1)

In [ ]:
inp = tf.keras.layers.Input((None, None,10))
x0 = tf.keras.layers.Conv2D(256,3,padding = 'same')(inp)
x = x0
for i in range(5):
    y = tf.keras.layers.Conv2D(256,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256, 1, activation = 'relu')(x)
for i in range(5):
    y = tf.keras.layers.Conv2D(256,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256,3,padding = 'same', activation = 'sigmoid')(x)
x = tf.keras.layers.Add()([x0,x])
x = tf.keras.layers.Conv2D(160,3,padding = 'same', activation = 'sigmoid')(x)
x = tf.keras.layers.Lambda (lambda z: tf.nn.depth_to_space(z, 4))(x)
x = tf.keras.layers.AveragePooling2D(4)(x)
out = x
model4_3 = tf.keras.models.Model(inputs = inp, outputs = out)

In [ ]:
inp = tf.keras.layers.Input((None, None,10))
x0 = tf.keras.layers.Conv2D(256,3,padding = 'same')(inp)
x = x0
for i in range(5):
    y = tf.keras.layers.Conv2D(256,3,activation = 'sigmoid',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256, 1, activation = 'sigmoid')(x)
for i in range(5):
    y = tf.keras.layers.Conv2D(256,3,activation = 'sigmoid',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256,3,padding = 'same', activation = 'sigmoid')(x)
x = tf.keras.layers.Add()([x0,x])
x = tf.keras.layers.Conv2D(160,3,padding = 'same', activation = 'sigmoid')(x)
x = tf.keras.layers.Lambda (lambda z: tf.nn.depth_to_space(z, 4))(x)
x = tf.keras.layers.AveragePooling2D(4)(x)
out = x
model4_4 = tf.keras.models.Model(inputs = inp, outputs = out)

In [ ]:
inp = tf.keras.layers.Input((None, None,10))
x0 = tf.keras.layers.Conv2D(256,3,padding = 'same')(inp)
x = x0
for i in range(5):
    y = tf.keras.layers.Conv2D(256,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
#x = tf.keras.layers.Conv2D(256, 1, activation = 'relu')(x)
for i in range(5):
    y = tf.keras.layers.Conv2D(256,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256,3,padding = 'same', activation = 'relu')(x)
x = tf.keras.layers.Add()([x0,x])
x = tf.keras.layers.Conv2D(160,3,padding = 'same', activation = 'hard_sigmoid')(x)
x = tf.keras.layers.Lambda (lambda z: tf.nn.depth_to_space(z, 4))(x)
x = tf.keras.layers.AveragePooling2D(4)(x)
out = x
model4_5_1 = tf.keras.models.Model(inputs = inp, outputs = out)

In [ ]:
print(model4_5_1.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 10)]                                                             
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   23296       ['input_1[0][0]']                
                                256)                                                              
                                                                                                  
 conv2d_1 (Conv2D)              (None, None, None,   590080      ['conv2d[0][0]']                 
                                256)                                                          

In [ ]:
inp = tf.keras.layers.Input((None, None,10))
x0 = tf.keras.layers.Conv2D(256,3,padding = 'same')(inp)
x = x0
for i in range(5):
    y = tf.keras.layers.Conv2D(256,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256, 1, activation = 'relu')(x)
for i in range(5):
    y = tf.keras.layers.Conv2D(256,3,activation = 'relu',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256,3,padding = 'same', activation = 'hard_sigmoid')(x)
x = tf.keras.layers.Add()([x0,x])
x = tf.keras.layers.Conv2D(160,3,padding = 'same', activation = 'hard_sigmoid')(x)
x = tf.keras.layers.Lambda (lambda z: tf.nn.depth_to_space(z, 4))(x)
x = tf.keras.layers.AveragePooling2D(4)(x)
out = x
model4_6 = tf.keras.models.Model(inputs = inp, outputs = out)

In [ ]:
inp = tf.keras.layers.Input((None, None,10))
x0 = tf.keras.layers.Conv2D(256,3,padding = 'same')(inp)
x = x0
for i in range(5):
    y = tf.keras.layers.Conv2D(256,3,activation = 'hard_sigmoid',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256, 1, activation = 'hard_sigmoid')(x)
for i in range(5):
    y = tf.keras.layers.Conv2D(256,3,activation = 'hard_sigmoid',padding = 'same')(x)
    y = tf.keras.layers.Conv2D(256,3,padding = 'same')(y)
    x = tf.keras.layers.Add()([x,y])
x = tf.keras.layers.Conv2D(256,3,padding = 'same', activation = 'hard_sigmoid')(x)
x = tf.keras.layers.Add()([x0,x])
x = tf.keras.layers.Conv2D(160,3,padding = 'same', activation = 'hard_sigmoid')(x)
x = tf.keras.layers.Lambda (lambda z: tf.nn.depth_to_space(z, 4))(x)
x = tf.keras.layers.AveragePooling2D(4)(x)
out = x
model4_7 = tf.keras.models.Model(inputs = inp, outputs = out)

In [ ]:
#model4_1.compile(loss = lssim, metrics = [mssim, psnr1], optimizer = optimizers.Adam(learning_rate = 1e-5))
#model4_2.compile(loss = lssim, metrics = [mssim, psnr1], optimizer = optimizers.Adam(learning_rate = 1e-5))
#model4_3.compile(loss = lssim, metrics = [mssim, psnr1], optimizer = optimizers.Adam(learning_rate = 1e-5))
model4_4.compile(loss = lssim, metrics = [mssim, psnr1], optimizer = optimizers.Adam(learning_rate = 1e-5))
model4_5.compile(loss = lssim, metrics = [mssim, psnr1], optimizer = optimizers.Adam(learning_rate = 1e-5))
model4_6.compile(loss = lssim, metrics = [mssim, psnr1], optimizer = optimizers.Adam(learning_rate = 1e-5))
model4_7.compile(loss = lssim, metrics = [mssim, psnr1], optimizer = optimizers.Adam(learning_rate = 1e-5))


In [ ]:
model4_5_1.compile(loss = lssim, metrics = [mssim, psnr1], optimizer = optimizers.Adam(learning_rate = 1e-5))


In [ ]:
model4_1.fit(train_dataset, epochs = 50, validation_data = valid_dataset, validation_freq = 10)

Epoch 1/50
125/125 [==============================] - 214s 2s/step - loss: 0.3219 - mssim: 0.6781 - psnr1: 10.8949
Epoch 2/50
125/125 [==============================] - 201s 2s/step - loss: 0.1518 - mssim: 0.8482 - psnr1: 17.8809
Epoch 3/50
125/125 [==============================] - 204s 2s/step - loss: 0.1200 - mssim: 0.8800 - psnr1: 22.4796
Epoch 4/50
125/125 [==============================] - 206s 2s/step - loss: 0.1114 - mssim: 0.8886 - psnr1: 23.3033
Epoch 5/50
125/125 [==============================] - 206s 2s/step - loss: 0.1067 - mssim: 0.8933 - psnr1: 23.7634
Epoch 6/50
125/125 [==============================] - 206s 2s/step - loss: 0.1041 - mssim: 0.8959 - psnr1: 23.8732
Epoch 7/50
125/125 [==============================] - 206s 2s/step - loss: 0.1014 - mssim: 0.8986 - psnr1: 24.2170
Epoch 8/50
125/125 [==============================] - 206s 2s/step - loss: 0.0991 - mssim: 0.9009 - psnr1: 24.4927
Epoch 9/50
125/125 [==============================] - 206s 2s/step - loss: 0.097

In [ ]:
model4_2.fit(train_dataset, epochs = 50, validation_data = valid_dataset, validation_freq = 10)

Epoch 1/50
125/125 [==============================] - 209s 2s/step - loss: 0.3329 - mssim: 0.6671 - psnr1: 11.3311
Epoch 2/50
125/125 [==============================] - 206s 2s/step - loss: 0.1547 - mssim: 0.8453 - psnr1: 18.0101
Epoch 3/50
125/125 [==============================] - 205s 2s/step - loss: 0.1215 - mssim: 0.8785 - psnr1: 22.4151
Epoch 4/50
125/125 [==============================] - 206s 2s/step - loss: 0.1127 - mssim: 0.8873 - psnr1: 23.1363
Epoch 5/50
125/125 [==============================] - 206s 2s/step - loss: 0.1080 - mssim: 0.8920 - psnr1: 23.5329
Epoch 6/50
125/125 [==============================] - 206s 2s/step - loss: 0.1044 - mssim: 0.8956 - psnr1: 23.8958
Epoch 7/50
125/125 [==============================] - 206s 2s/step - loss: 0.1021 - mssim: 0.8979 - psnr1: 24.0594
Epoch 8/50
125/125 [==============================] - 206s 2s/step - loss: 0.1004 - mssim: 0.8996 - psnr1: 24.1445
Epoch 9/50
125/125 [==============================] - 206s 2s/step - loss: 0.098

In [ ]:
model4_3.fit(train_dataset, epochs = 50, validation_data = valid_dataset, validation_freq = 10)

Epoch 1/50
125/125 [==============================] - 213s 2s/step - loss: 0.4161 - mssim: 0.5839 - psnr1: 13.5729
Epoch 2/50
125/125 [==============================] - 207s 2s/step - loss: 0.1936 - mssim: 0.8064 - psnr1: 17.7439
Epoch 3/50
125/125 [==============================] - 208s 2s/step - loss: 0.1511 - mssim: 0.8489 - psnr1: 20.5059
Epoch 4/50
125/125 [==============================] - 208s 2s/step - loss: 0.1344 - mssim: 0.8656 - psnr1: 21.8254
Epoch 5/50
125/125 [==============================] - 208s 2s/step - loss: 0.1253 - mssim: 0.8747 - psnr1: 22.4543
Epoch 6/50
125/125 [==============================] - 209s 2s/step - loss: 0.1191 - mssim: 0.8809 - psnr1: 22.9187
Epoch 7/50
125/125 [==============================] - 208s 2s/step - loss: 0.1144 - mssim: 0.8856 - psnr1: 23.3067
Epoch 8/50
125/125 [==============================] - 208s 2s/step - loss: 0.1107 - mssim: 0.8893 - psnr1: 23.6387
Epoch 9/50
125/125 [==============================] - 208s 2s/step - loss: 0.108

In [ ]:
model4_4.fit(train_dataset, epochs = 50, validation_data = valid_dataset, validation_freq = 10)

Epoch 1/50
125/125 [==============================] - 215s 2s/step - loss: 0.5994 - mssim: 0.4006 - psnr1: 13.9368
Epoch 2/50
125/125 [==============================] - 198s 2s/step - loss: 0.3008 - mssim: 0.6992 - psnr1: 17.1889
Epoch 3/50
125/125 [==============================] - 198s 2s/step - loss: 0.2113 - mssim: 0.7887 - psnr1: 19.1390
Epoch 4/50
125/125 [==============================] - 198s 2s/step - loss: 0.1842 - mssim: 0.8158 - psnr1: 19.8946
Epoch 5/50
125/125 [==============================] - 199s 2s/step - loss: 0.1707 - mssim: 0.8293 - psnr1: 20.1798
Epoch 6/50
125/125 [==============================] - 198s 2s/step - loss: 0.1623 - mssim: 0.8377 - psnr1: 20.4308
Epoch 7/50
125/125 [==============================] - 199s 2s/step - loss: 0.1559 - mssim: 0.8441 - psnr1: 20.6303
Epoch 8/50
125/125 [==============================] - 199s 2s/step - loss: 0.1507 - mssim: 0.8493 - psnr1: 20.8472
Epoch 9/50
125/125 [==============================] - 198s 2s/step - loss: 0.146

In [ ]:
model4_5.fit(train_dataset, epochs = 50, validation_data = valid_dataset, validation_freq = 10)

Epoch 1/50
125/125 [==============================] - 202s 2s/step - loss: 0.3063 - mssim: 0.6937 - psnr1: 10.3380
Epoch 2/50
125/125 [==============================] - 196s 2s/step - loss: 0.1510 - mssim: 0.8490 - psnr1: 16.2362
Epoch 3/50
125/125 [==============================] - 197s 2s/step - loss: 0.1161 - mssim: 0.8839 - psnr1: 22.5299
Epoch 4/50
125/125 [==============================] - 196s 2s/step - loss: 0.1086 - mssim: 0.8914 - psnr1: 23.4237
Epoch 5/50
125/125 [==============================] - 197s 2s/step - loss: 0.1041 - mssim: 0.8959 - psnr1: 23.9748
Epoch 6/50
125/125 [==============================] - 197s 2s/step - loss: 0.1014 - mssim: 0.8986 - psnr1: 24.2226
Epoch 7/50
125/125 [==============================] - 196s 2s/step - loss: 0.0991 - mssim: 0.9009 - psnr1: 24.4759
Epoch 8/50
125/125 [==============================] - 196s 2s/step - loss: 0.0974 - mssim: 0.9026 - psnr1: 24.6594
Epoch 9/50
125/125 [==============================] - 196s 2s/step - loss: 0.095

In [ ]:
model4_5_1.fit(train_dataset, epochs = 50, validation_data = valid_dataset, validation_freq = 10)

Epoch 1/50
125/125 [==============================] - 199s 2s/step - loss: 0.3033 - mssim: 0.6967 - psnr1: 10.8964
Epoch 2/50
125/125 [==============================] - 194s 2s/step - loss: 0.1488 - mssim: 0.8512 - psnr1: 16.7597
Epoch 3/50
125/125 [==============================] - 194s 2s/step - loss: 0.1156 - mssim: 0.8844 - psnr1: 22.3513
Epoch 4/50
125/125 [==============================] - 194s 2s/step - loss: 0.1072 - mssim: 0.8928 - psnr1: 23.4801
Epoch 5/50
125/125 [==============================] - 194s 2s/step - loss: 0.1025 - mssim: 0.8975 - psnr1: 24.0821
Epoch 6/50
125/125 [==============================] - 195s 2s/step - loss: 0.0996 - mssim: 0.9004 - psnr1: 24.4167
Epoch 7/50
125/125 [==============================] - 193s 2s/step - loss: 0.0975 - mssim: 0.9025 - psnr1: 24.6281
Epoch 8/50
125/125 [==============================] - 194s 2s/step - loss: 0.0958 - mssim: 0.9042 - psnr1: 24.7794
Epoch 9/50
125/125 [==============================] - 194s 2s/step - loss: 0.094

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue May 24 14:42:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    31W /  70W |    326MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model4_5_1.save('drive/MyDrive/models/final')

INFO:tensorflow:Assets written to: drive/MyDrive/models/final/assets


In [ ]:
model4_6.fit(train_dataset, epochs = 50, validation_data = valid_dataset, validation_freq = 10)

Epoch 1/50
125/125 [==============================] - 203s 2s/step - loss: 0.4299 - mssim: 0.5701 - psnr1: 12.6292
Epoch 2/50
125/125 [==============================] - 197s 2s/step - loss: 0.1943 - mssim: 0.8057 - psnr1: 16.5590
Epoch 3/50
125/125 [==============================] - 198s 2s/step - loss: 0.1443 - mssim: 0.8557 - psnr1: 20.5841
Epoch 4/50
125/125 [==============================] - 197s 2s/step - loss: 0.1273 - mssim: 0.8727 - psnr1: 22.2075
Epoch 5/50
125/125 [==============================] - 197s 2s/step - loss: 0.1193 - mssim: 0.8807 - psnr1: 22.8036
Epoch 6/50
125/125 [==============================] - 198s 2s/step - loss: 0.1144 - mssim: 0.8856 - psnr1: 23.1815
Epoch 7/50
125/125 [==============================] - 199s 2s/step - loss: 0.1103 - mssim: 0.8897 - psnr1: 23.5134
Epoch 8/50
125/125 [==============================] - 199s 2s/step - loss: 0.1073 - mssim: 0.8927 - psnr1: 23.7702
Epoch 9/50
125/125 [==============================] - 199s 2s/step - loss: 0.105

In [ ]:
model4_7.fit(train_dataset, epochs = 50, validation_data = valid_dataset, validation_freq = 10)

Epoch 1/50
125/125 [==============================] - 213s 2s/step - loss: 0.6159 - mssim: 0.3841 - psnr1: 13.3754
Epoch 2/50
125/125 [==============================] - 207s 2s/step - loss: 0.3518 - mssim: 0.6482 - psnr1: 16.0855
Epoch 3/50
125/125 [==============================] - 207s 2s/step - loss: 0.2112 - mssim: 0.7888 - psnr1: 18.9482
Epoch 4/50
125/125 [==============================] - 208s 2s/step - loss: 0.1774 - mssim: 0.8226 - psnr1: 20.2337
Epoch 5/50
125/125 [==============================] - 208s 2s/step - loss: 0.1663 - mssim: 0.8337 - psnr1: 20.5910
Epoch 6/50
125/125 [==============================] - 207s 2s/step - loss: 0.1601 - mssim: 0.8399 - psnr1: 20.6668
Epoch 7/50
125/125 [==============================] - 208s 2s/step - loss: 0.1546 - mssim: 0.8454 - psnr1: 20.7462
Epoch 8/50
125/125 [==============================] - 208s 2s/step - loss: 0.1495 - mssim: 0.8505 - psnr1: 20.9103
Epoch 9/50
125/125 [==============================] - 208s 2s/step - loss: 0.145